# Analiza wariantów

In [ ]:
# sprawdzenie czy dane są dostępne
! gsutil ls gs://edugen-lab-agaszmurlo/vcf/


## Przygotowanie sesji Spark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession \
.builder \
.config('spark.driver.memory','1g') \
.config('spark.executor.memory', '2g') \
.getOrCreate()

In [ ]:
import os                               # moduł OS języka Python
user_name = os.environ.get('USER')      # pobieramy zmienną środowiskową USER
bucket = f"gs://edugen-lab-{user_name}" # konstruujemy sciezke dostepowa do pliku
print(bucket)

## Stworzenie tabeli z wariantami

In [ ]:
var_path=f"{bucket}/vcf/mother.vcf"
table_var = 'variants'

spark.sql(f'DROP TABLE IF EXISTS {table_var}')

spark.sql(f'CREATE TABLE IF NOT EXISTS {table_var} \
USING org.biodatageeks.sequila.datasources.VCF.VCFDataSource \
OPTIONS(path "{var_path}")')

## Weryfikacja danych

## <span style='background:yellow'> ZADANIE  </span>


Napisz polecenie, które pokaże strukturę (kolumny i ich typy) tabeli z wariantami

In [ ]:
spark.sql(f'describe {table_var}').show(40, truncate=False)

In [ ]:
spark.sql(f'select count (*) from {table_var}').show()

Widzimy, że ALT jest rodzaju tablicowego, sprawdźmy, czy są warianty mutlialleliczne?

In [ ]:
spark.sql(f'select * from {table_var} where size(alt) > 1').show()

In [ ]:
spark.sql(f'select * from {table_var} where size(alt) > 1').count() # ile ich jest ? 

In [ ]:
spark.sql(f'select genotypes from {table_var}').show(truncate=False) # tabela w postaci nieznormalizowanej -> mamy listę wartości